In [7]:
%load_ext autoreload
%autoreload 2
from importlib import reload
import glob
import pickle
import numpy as np
import os
import astropy.io.fits as fits
import pandas as pd
import pyklip.klip as klip
import pyklip.instruments.MagAO as MagAO
import pyklip.parallelized as parallelized
import pyklip.fakes as fakes
import matplotlib.pyplot as plt
import snr
import sys
import pdb
from tqdm import tqdm
import seaborn as sns
import collapse as c

#####################
# Display and backend
#####################
plt.style.use("default") # Reset to default before layering on any changes
FIG_LARGE = (12, 8) # Default plot figure size for large figures
%config InlineBackend.figure_format = "retina" # Crisp retina display on macs
#%matplotlib qt5 

################
# Theme settings
################
notebook_mode = "paper" # (paper | dark)
sns.set(palette="colorblind", color_codes=True, context="talk")

if notebook_mode.lower() == "paper":
    tick_params = {
        "xtick.top":True,
        "xtick.direction":"in",
        "ytick.right":True,
        "ytick.direction":"in",
        "axes.spine.right":True
    }
    sns.set_style("ticks", tick_params)
    params = {
        "axes.formatter.limits":(-3, 7),
        "xtick.major.size":14,
        "ytick.major.size":14,
        "xtick.minor.visible":True,
        "ytick.minor.visible":True,
    }
    plt.rcParams.update(params)

elif notebook_mode.lower() == "dark":
    plt.style.use("cyberpunk")

else:
    plt.style.use("default")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
